In [1]:
import networkx as nx
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
f = pd.read_csv('PP-Pathways_ppi.csv', header = None)

In [3]:
f

,0,1
0,1394,2778
1,6331,17999
2,122704,54460
3,2597,2911
4,4790,79155
...,...,...
342348,10969,4670
342349,6774,55748
342350,23299,399693
342351,9247,337967


In [31]:
G = nx.from_pandas_edgelist(f,  0,  1)

In [32]:
sub, _ = train_test_split(f, random_state = 0, train_size = 0.1)

In [33]:
sub

,0,1
212386,5565,4223
299313,200081,672
174066,27306,5740
260147,8517,353
248564,218885,3949
...,...,...
122579,7417,55967
304137,286077,23623
152315,58477,7444
117952,5795,63978


In [34]:
train, test = train_test_split(sub, random_state = 0, train_size = 0.8, test_size = 0.2)

In [35]:
temp = test.values.tolist()
testset = []
for each in temp:
    if each[0] != each[1]:
        testset.append(each)

In [36]:
G = nx.from_pandas_edgelist(sub,  0,  1)
for each in testset:
    G.remove_edge(each[0], each[1])

In [37]:
preds = nx.common_neighbor_centrality(G, testset)

In [38]:
predicted = []

In [39]:
for u, v, p in preds:
    if p > 0:
        predicted.append((u,v,p))
    


In [40]:
predicted

[(1027, 1020, 637.6999999999998),
 (90957, 386674, 637.6999999999998),
 (178, 2739, 510.1599999999999),
 (3603, 57594, 850.2666666666664),
 (51605, 83660, 425.1333333333332),
 (8976, 94134, 425.1333333333332),
 (23641, 170082, 637.6999999999998),
 (55604, 832, 850.2666666666664),
 (7718, 10768, 850.2666666666664),
 (2885, 10915, 850.2666666666664),
 (2296, 5089, 850.2666666666664),
 (2335, 3009, 850.2666666666664),
 (2130, 4879, 850.2666666666664),
 (3091, 8554, 850.2666666666664),
 (3053, 3493, 425.1333333333332),
 (3068, 8362, 850.2666666666664),
 (51259, 10483, 510.1599999999999),
 (11007, 148223, 850.2666666666664),
 (320, 8825, 510.1599999999999),
 (2099, 5268, 637.6999999999998),
 (2309, 7533, 850.2666666666664),
 (6693, 29978, 637.6999999999998),
 (2335, 4171, 1276.9999999999995),
 (58517, 6625, 1276.9999999999995),
 (10401, 3065, 850.2666666666664),
 (114928, 22980, 425.1333333333332),
 (4609, 3091, 1276.1999999999996),
 (5883, 983, 850.2666666666664),
 (25489, 156, 637.6999999